# LanceDB 임베딩 생성 (Google Colab)

Google Colab T4 GPU 환경에서 법령/판례 임베딩을 생성합니다.

## 환경
- GPU: T4 (15GB VRAM)
- RAM: 12-25GB
- 권장 배치 크기: 64

## 주의사항
- Colab 무료 버전은 런타임이 12시간 후 끊길 수 있음
- Google Drive에 체크포인트를 저장하여 중단 시 재개 가능

## 1. 패키지 설치

In [ ]:
!pip install lancedb sentence-transformers pyarrow ijson psutil tqdm gdown -q
print("패키지 설치 완료")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정 (Drive에 저장하면 런타임 끊겨도 유지됨)
import os
WORK_DIR = '/content/drive/MyDrive/law-embeddings'
os.makedirs(WORK_DIR, exist_ok=True)
print(f"작업 디렉토리: {WORK_DIR}")

## 3. 데이터 준비

Google Drive에 미리 업로드하거나, 공유 링크에서 다운로드합니다.

In [ ]:
import os

# 방법 A: Drive에 이미 업로드된 경우
DATA_DIR = '/content/drive/MyDrive/law-data'
LAW_PATH = os.path.join(DATA_DIR, 'law_cleaned.json')
PREC_PATH = os.path.join(DATA_DIR, 'precedents_cleaned.json')

# 방법 B: gdown으로 다운로드
# LAW_FILE_ID = 'YOUR_LAW_FILE_ID'
# PREC_FILE_ID = 'YOUR_PRECEDENT_FILE_ID'
# !gdown --id {LAW_FILE_ID} -O {LAW_PATH}
# !gdown --id {PREC_FILE_ID} -O {PREC_PATH}

# 파일 확인
for path in [LAW_PATH, PREC_PATH]:
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  {os.path.basename(path)}: {size_mb:.1f} MB")
    else:
        print(f"  {os.path.basename(path)}: NOT FOUND")

## 4. 스크립트 로드

In [ ]:
# 스크립트를 Drive에서 복사하거나 직접 업로드
SCRIPT_PATH = os.path.join(WORK_DIR, 'runpod_lancedb_embeddings.py')
if os.path.exists(SCRIPT_PATH):
    exec(open(SCRIPT_PATH).read())
    print("스크립트 로드 완료")
else:
    print(f"스크립트를 {SCRIPT_PATH}에 업로드해주세요")
    print("또는 Colab 파일 탭에서 runpod_lancedb_embeddings.py를 업로드하세요")

In [ ]:
# 디바이스 확인
device_info, config = print_device_info()
print_memory_status()

## 5. LanceDB 저장 경로 설정 (Drive)

런타임 끊김 대비 Drive에 직접 저장합니다.

In [ ]:
import os

# LanceDB 데이터를 Drive에 저장
LANCEDB_PATH = os.path.join(WORK_DIR, 'lancedb_data')
os.makedirs(LANCEDB_PATH, exist_ok=True)

# 환경변수 설정
os.environ['LANCEDB_URI'] = LANCEDB_PATH
print(f"LanceDB 저장 경로: {LANCEDB_PATH}")

# CONFIG 업데이트
CONFIG['LANCEDB_URI'] = LANCEDB_PATH

## 6. 판례 분할 처리 (권장)

65,000건 판례를 한번에 처리하면 Colab 메모리 초과 위험이 있습니다.
5,000건씩 분할하여 처리합니다.

In [ ]:
# 판례 분할 (Drive에 저장)
SPLIT_DIR = os.path.join(WORK_DIR, 'split_data')
os.makedirs(SPLIT_DIR, exist_ok=True)

part_files = split_precedents(
    PREC_PATH,
    chunk_size=5000,
    output_dir=SPLIT_DIR,
)
print(f"분할 완료: {len(part_files)} 파일")

## 7. 법령 임베딩

In [ ]:
# 법령 임베딩 (T4: batch_size=64 권장)
law_stats = run_law_embedding(
    LAW_PATH,
    reset=True,
    batch_size=64,
)
print(f"\n법령 완료: {law_stats}")
print_memory_status()

## 8. 판례 임베딩 (분할 파일)

In [ ]:
import glob

# 분할된 판례 파일 전체 처리
pattern = os.path.join(SPLIT_DIR, 'precedents_part_*.json')
total_stats = run_all_precedent_parts(
    pattern,
    batch_size=64,
)
print(f"\n판례 전체 완료: {total_stats}")

## 9. 최종 통계 및 검증

In [ ]:
show_stats()
print_memory_status()

# 검색 테스트
import lancedb
model = get_embedding_model()
query_vector = model.encode('손해배상 책임')

db = lancedb.connect(LANCEDB_PATH)
table = db.open_table('legal_chunks')
results = table.search(query_vector).metric('cosine').limit(5).to_pandas()

print(f"\n검색 테스트: '손해배상 책임'")
for _, row in results.iterrows():
    sim = 1 - row['_distance']
    print(f"  [{sim:.4f}] [{row['data_type']}] {row['title'][:50]}")

## 10. 결과 다운로드

Drive에 이미 저장되어 있으므로 ZIP으로 압축하여 로컬에 다운로드합니다.

In [ ]:
# ZIP 압축 (Drive에 저장)
ZIP_PATH = os.path.join(WORK_DIR, 'lancedb_data.zip')
!cd {WORK_DIR} && zip -r lancedb_data.zip lancedb_data/

import os
size_mb = os.path.getsize(ZIP_PATH) / (1024 * 1024)
print(f"\n{ZIP_PATH}: {size_mb:.1f} MB")
print("Google Drive에서 직접 다운로드하세요")
print(f"경로: My Drive/law-embeddings/lancedb_data.zip")

In [ ]:
# 메모리 정리
clear_model_cache()
clear_memory()
print("정리 완료")